# Reproduce test data predictions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pickle
import tensorflow
from utils import *
from tqdm.notebook import tqdm
from ipywidgets import *

In [ ]:
#load test data
test_data_finite = np.load('Data/test data invscid burgers finiteNet.npy') 
test_data_weno5 = np.load('Data/test data invscid burgers weno5.npy')
test_data_exact = np.load('Data/test data invscid burgers exact.npy') 

#load model
loaded_model = tensorflow.keras.models.load_model('SavedModels/burger std=0.7968797813840656 mean=1.4804453986909523 loss=0.0019.h5' ,custom_objects={'LeakyReLU':tensorflow.keras.layers.LeakyReLU()})
std  = 0.7968797813840656 ; mean = 1.4804453986909523

In [ ]:
chunks_slider = IntRangeSlider(value=[0, 0],min=0,max=9,step=1,description='Chunks:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)
tests_slider = IntSlider(value=100,min=0,max=1000,step=1,description='Tests#:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)

@interact(chunks=chunks_slider,tests = tests_slider,start=False)
def test_model(chunks,tests,start):
    std  = 0.7968797813840656 ; mean = 1.4804453986909523
    chunks = np.arange(chunks[0],chunks[1]+1)
    
    if start : 
        
        prediction_solution = np.zeros((tests,101,12,12,1))
        
        for chunk in (chunks):

            chunk_indices = np.arange(chunk,101,10)

            print(f'Chunk = {chunk}\nInput snapshot at {chunk_indices[0]} \nPredicted snapshots at {chunk_indices[1:]}')

            for test in tqdm(range(tests)):

                #preprocess input
                test_input = ( test_data_weno5[[test],chunk_indices[0]].reshape(1,1,10,10,1) - mean ) /std
                padded_test_input = np.zeros((1,1,12,12,1))
                padded_test_input[:,:,1:-1,1:-1,:] = test_input 
                test_prediction = recursive_prediction(loaded_model,padded_test_input,10,12)

                #place the prediction at the appropriate test index and time index 
                prediction_solution[test:test+1,chunk_indices[1:]] = test_prediction[:,:len(chunk_indices[1:])]
        
        np.save('Data/test data invscid burgers DiffusionNet.npy' , (prediction_solution[:,:,1:-1,1:-1,:].reshape(tests,101,100) * std ) + mean )
        
        print('Saved...')